## Colmap data testing for tufts turf images

In [1]:
# Tufts turf field located near 


import open3d as o3d
import numpy as np
import plotly.graph_objects as go 
from scipy.spatial.transform import Rotation as R
from colmapParsingUtils import *
import copy

from ptCloudParsingUtils import *


from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.pyplot as plt

import tensorflow as tf
import cv2

%load_ext autoreload
%autoreload 2
%autosave 180


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2025-06-10 10:34:32.761204: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-10 10:34:32.769307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 10:34:32.779074: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 10:34:32.782052: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 10:34:32.789762: I tensorflow/core/platform/cpu_feature_guar

Autosaving every 180 seconds


In [2]:
# Load images, points, pose estimations from COLMAP 

images_colmap = read_images_text('/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/images.txt')
cameras = read_cameras_text('/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/cameras.txt')
pts3d = read_points3D_text('/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/points3D.txt')

In [3]:
# Extract 3d points from raw file 

pointdata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/points3D.txt"
num_pts = 75497 # Gerrard Hall

# points_data = []
scene_pts = np.zeros([num_pts, 3]) 
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:]  # Skip the first three lines
    
    for line in lines:
        elements = line.strip().split()
        
        row = [float(elem) for elem in list(elements)]

        scene_pts[idx,0] = row[1]
        scene_pts[idx,1] = row[2]
        scene_pts[idx,2] = row[3]
        idx += 1

# Filter RGB data into separate numpy array 
rgb_data = np.zeros([num_pts, 3])
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:] # Skip the first three lines 
    for line in lines:
        elements = line.strip().split()
        row = [float(elem) for elem in list(elements)]

        rgb_data[idx,0] = row[4]
        rgb_data[idx,1] = row[5]
        rgb_data[idx,2] = row[6]
        idx += 1

# SCALE COLORS
rgb_data = rgb_data/255
# print(scene_pts[:10])

# Filter RGB data into separate numpy array with ALPHA
rgba_data = np.zeros([num_pts, 4])
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:] # Skip the first three lines 
    for line in lines:
        elements = line.strip().split()
        row = [float(elem) for elem in list(elements)]

        rgba_data[idx,0] = row[4]
        rgba_data[idx,1] = row[5]
        rgba_data[idx,2] = row[6]
        rgba_data[idx,3] = .5
        idx += 1

# SCALE COLORS
# rgba_data = rgba_data/255
rgba_data[:, :3] = rgba_data[:, :3] / 255
# print(scene_pts[:10])

In [8]:
# Convert COLMAP poses (xyz,quats) to rotm

poses = np.zeros([len(images_colmap),4,4])
images = np.zeros([len(poses),250,250,3])

print(images_colmap)
skip_indices = {10, 17, 65, 83, 84}  # Use a set to store the indices to skip

#loop through <images_from_colmap> to get 3D poses of cameras at each timestamp
# print(len(images_from_colmap))
for n in range(len(images_colmap)):
    # print(n)
    if n in skip_indices:
        continue


    # Pull quaternion and translation vector
    qvec = images_colmap[n+1].qvec #raw
    # print(qvec)
    # print(images_colmap[n+1].tvec)
    tvec = images_colmap[n+1].tvec[:,None]
    # print(tvec)
    
    t = tvec.reshape([3,1])
    # print(tvec)
    Rotmat = qvec2rotmat(-qvec)
    
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    m = np.concatenate([np.concatenate([Rotmat, t], 1), bottom], 0)
    c2w = np.linalg.inv(m)

    # NERF SPECIFIC ROTATIONS 
    # c2w[0:3,2] *= -1 # flip the y and z axis
    # c2w[0:3,1] *= -1
    # c2w = c2w[[1,0,2,3],:]
    # c2w[2,:] *= -1 # flip whole world upside down
    


    poses[n] = c2w
    # poses[n] = m
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#GET REST OF PARAMS NEEDED FOR tinyNeRF format~~~~~~~~~~~~~~~~~~~~~~~~~~~~    

#fix order of colors
images[:,:,:,0], images[:,:,:,1] = images[:,:,:,1], images[:,:,:,0]

H,W = images.shape[1:3]
# print(H,W)
testimg, testpose = images[55], poses[55]

focal = cameras[1].params[0] #test- see if same focal length can be shared across all images
focal = focal/12
print(focal)
# print(poses)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

{147: Image(id=147, qvec=array([ 0.94682736,  0.16635069,  0.09165848, -0.25970008]), tvec=array([-0.58608619, -2.45403803,  3.78283416]), camera_id=1, name='IMG_9589.JPEG', xys=array([[1572.97473145,    4.59492731],
       [3845.20898438,    6.86619711],
       [3356.97924805,   10.33217144],
       ...,
       [1107.99316406, 1426.48974609],
       [2349.85375977, 1716.33422852],
       [1731.72827148,  921.21459961]]), point3D_ids=array([-1, -1, -1, ..., -1, -1, -1])), 146: Image(id=146, qvec=array([ 0.95611918,  0.00819863,  0.03998516, -0.2901208 ]), tvec=array([-0.69285362, -1.02334269,  4.43570543]), camera_id=1, name='IMG_9588.JPEG', xys=array([[3328.92919922,    6.82591105],
       [3328.92919922,    6.82591105],
       [3964.43994141,    7.4608655 ],
       ...,
       [ 359.3979187 , 1507.11608887],
       [1433.13305664, 1612.39819336],
       [1047.85473633, 2528.19970703]]), point3D_ids=array([-1, -1, -1, ..., -1, -1, -1])), 145: Image(id=145, qvec=array([ 0.85164569, -0.

In [9]:
print(poses)

[[[ 0.85588138 -0.51712694 -0.00684059  0.55223632]
  [ 0.51306884  0.8473535   0.13693946  1.5193403 ]
  [-0.06501869 -0.12071363  0.9905558  -5.7187042 ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.7877738  -0.55153047  0.27427466  0.60683138]
  [ 0.61542695  0.68615026 -0.38787561  1.40580374]
  [ 0.02573159  0.47435426  0.87995791 -5.67667271]
  [ 0.          0.          0.          1.        ]]

 [[ 0.84914521 -0.52713696 -0.03284869  0.62900158]
  [ 0.52744921  0.84313725  0.10448398  1.26933531]
  [-0.02738142 -0.10604809  0.99398393 -5.39971041]
  [ 0.          0.          0.          1.        ]]

 ...

 [[ 0.89172914 -0.30503252 -0.33432663  2.08995241]
  [ 0.44760737  0.70353018  0.55198998 -2.51182197]
  [ 0.06683398 -0.64187262  0.76389316 -1.85583248]
  [ 0.          0.          0.          1.        ]]

 [[ 0.93267526 -0.05066988 -0.35714062  1.47761334]
  [ 0.34367728  0.42553304  0.8371425  -1.0484954 ]
  [ 0.10955722 -0.90352321  0.41429822 -2.97099372

# Plotting pt cloud

In [6]:
# Plotting 3D points
# BY ITSELF
# Data generated by COLMAP

vis = o3d.visualization.Visualizer()
vis.create_window()

scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgba_data)
vis.add_geometry(scene_cloud)

# render_option = vis.get_render_option()
render_option = vis.get_render_option()

render_option.point_size = 2.5
# render_option.point_alpha = .5

vis.run()

vis.destroy_window()


libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
pci id for fd 75: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


# Plot camera poses 

In [13]:
print(poses)

[[[ 0.85588138 -0.51712694 -0.00684059  0.55223632]
  [ 0.51306884  0.8473535   0.13693946  1.5193403 ]
  [-0.06501869 -0.12071363  0.9905558  -5.7187042 ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.7877738  -0.55153047  0.27427466  0.60683138]
  [ 0.61542695  0.68615026 -0.38787561  1.40580374]
  [ 0.02573159  0.47435426  0.87995791 -5.67667271]
  [ 0.          0.          0.          1.        ]]

 [[ 0.84914521 -0.52713696 -0.03284869  0.62900158]
  [ 0.52744921  0.84313725  0.10448398  1.26933531]
  [-0.02738142 -0.10604809  0.99398393 -5.39971041]
  [ 0.          0.          0.          1.        ]]

 ...

 [[ 0.89172914 -0.30503252 -0.33432663  2.08995241]
  [ 0.44760737  0.70353018  0.55198998 -2.51182197]
  [ 0.06683398 -0.64187262  0.76389316 -1.85583248]
  [ 0.          0.          0.          1.        ]]

 [[ 0.93267526 -0.05066988 -0.35714062  1.47761334]
  [ 0.34367728  0.42553304  0.8371425  -1.0484954 ]
  [ 0.10955722 -0.90352321  0.41429822 -2.97099372

In [12]:
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud', width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)

# loop through poses 
for pose in poses:
    homog_t = pose
    print(homog_t)
    axes1 = copy.deepcopy(axes).transform(homog_t)
    vis.add_geometry(axes1)
    

vis.add_geometry(axes)
# vis.add_geometry(scene_cloud)

# Run the visualizer
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.


[[ 0.85588138 -0.51712694 -0.00684059  0.55223632]
 [ 0.51306884  0.8473535   0.13693946  1.5193403 ]
 [-0.06501869 -0.12071363  0.9905558  -5.7187042 ]
 [ 0.          0.          0.          1.        ]]
[[ 0.7877738  -0.55153047  0.27427466  0.60683138]
 [ 0.61542695  0.68615026 -0.38787561  1.40580374]
 [ 0.02573159  0.47435426  0.87995791 -5.67667271]
 [ 0.          0.          0.          1.        ]]
[[ 0.84914521 -0.52713696 -0.03284869  0.62900158]
 [ 0.52744921  0.84313725  0.10448398  1.26933531]
 [-0.02738142 -0.10604809  0.99398393 -5.39971041]
 [ 0.          0.          0.          1.        ]]
[[ 0.79304202 -0.56241779  0.23403116  0.65248268]
 [ 0.60527117  0.6841197  -0.40697304  1.15045675]
 [ 0.06878355  0.46439903  0.88295094 -5.21272475]
 [ 0.          0.          0.          1.        ]]
[[ 0.83714628 -0.53943088 -0.09055629  0.63299746]
 [ 0.54314     0.83937992  0.0209834   1.15556217]
 [ 0.06469203 -0.06675092  0.99567025 -5.13870638]
 [ 0.          0.          

using driver i915 for 82
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 82
pci id for fd 82: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


# Frames and cloud 

In [ ]:
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud', width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# loop through poses 
for pose in poses:
    homog_t = pose
    axes1 = copy.deepcopy(axes).transform(homog_t)
    vis.add_geometry(axes1)
    

vis.add_geometry(axes)
vis.add_geometry(scene_cloud)

# Run the visualizer
vis.run()
vis.destroy_window()